<a href="https://colab.research.google.com/github/Snehansh1/RAG/blob/master/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = 'your_api_key'

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=['url'])

docs = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

splits = text_splitter.split_documents(docs)

In [8]:
print(len(splits))

11


In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma

embedding = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [11]:
print(vectorstore._collection.count())

11


In [12]:
print(vectorstore._collection.get())

{'ids': ['5bdf2644-d059-4219-a551-e0ea9e99f30d', 'ec80f026-5434-4472-a259-5e5c3f57be59', 'd10b21fc-5fad-4732-a480-47a162e78546', '5adc28fc-55b3-4751-903c-69b54b362e56', '53dc6174-6594-46f1-88ce-995eb4077176', 'abfb79c8-394f-4ef1-80c4-170accf5dfb3', '286ea7ac-acb1-4205-bc27-6167d312baa5', '4ecd04ba-774a-4b90-8e96-5a5e0ed9b031', 'a9192026-01f8-4f31-8d26-513b07172daa', 'cd06e48b-b954-4691-b457-a7c8a72887ec', '688894e8-8e8a-45ad-88a6-41d5049c12af'], 'embeddings': None, 'documents': ['Hands-on Generative AI CourseCoursesBundle CoursesMentorFree ContentTestimonialsFAQLogin Signup Starts on 16th September 2025Hands-on Generative AI CourseLearn, Build, Deploy and Apply Generative AI7 weeks · 3 classes/week · 2 hrs/class + Post-class Doubt SupportClasses on Tue, Wed, Thurs - 9PM ISTAccess all Live BatchesLifetime access of RecordingsAccess Discord CommunityCode availableBuild ProjectsLearn Future-Ready TechEnroll 1Week 1Foundations of Generative AI Introduction to AI Mathematical Foundations fo

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [22]:
def format_docs(docs):
  return "\n".join(doc.page_content for doc in docs)

In [25]:
rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}
             | prompt
             | llm
             | StrOutputParser())

In [26]:
rag_chain.invoke("Are the recordings of the course available? For how long?")

'Yes, recordings of the course are available. You will have lifetime access to these recordings.'

In [27]:
rag_chain.invoke("Are the testimonials for the course available?")

'Yes, testimonials for the course are available. The context includes several testimonials from individuals such as Sudarshan Suresh Srikant, Syed I, and Manika Kaushik. These testimonials praise the course content, the instructor Keerti, and the practical application of the knowledge gained.'